In [74]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import ast
from IPython.display import clear_output

In [75]:
# Steg 1: Ladda dataset
original_training_data = pd.read_csv("training_knn.csv")
exercise_data = pd.read_csv('gym.csv', delimiter=';')
important_data = exercise_data[['Exercise Name', 'Main_muscle', 'Mechanics', 'Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Difficulty (1-5)', 'Secondary Muscles']].copy()
encode_columns = ['Main_muscle', 'Mechanics','Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Secondary Muscles']
ID_columns = ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", "ID_Secondary Muscles"]

important_data.head()

,Exercise Name,Main_muscle,Mechanics,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Difficulty (1-5),Secondary Muscles
0,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
1,Military Press,Shoulder,Compound,"Anterior Deltoid,","Pectoralis Major, Clavicular, Triceps Brachii,...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head, Biceps Brachii, Short Head,",4,"Lateral Deltoid, Triceps Brachii"
2,Military Press: Seated,Shoulder,Compound,"Anterior Deltoid,","Deltoid, Lateral, Supraspinatus, Triceps Brach...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head,",3,"Lateral Deltoid, Triceps Brachii"
3,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
4,Front Raise: Alternating,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"


In [76]:
def encoded_data(specific_muscle):
    new_data = important_data.copy()
    target_muscle_mapping = []  # List to store muscle names at correct indices
    
    for column in encode_columns:
        new_data.loc[:, column] = new_data[column].fillna("")
        new_data.loc[:, column] = new_data[column].apply(lambda x: x.split(", ") if x else [])
        
        # Multi-label binarization
        mlb = MultiLabelBinarizer()
        binary_matrix = mlb.fit_transform(new_data[column])
        
        # Save muscle names in a list if the column is "Target_Muscles"
        if column == "Target_Muscles":
            target_muscle_mapping = list(mlb.classes_)        
        new_data.loc[:, f"ID_{column}"] = binary_matrix.tolist()
        new_data = new_data.drop(columns=[column])
    finished_data = pd.DataFrame()
    for i in range(len(target_muscle_mapping)):
        muscle_index=[]
        if target_muscle_mapping[i].replace(",","") == specific_muscle:
            muscle_index.append(i)
        for index in muscle_index:
            temporary_dataset = new_data[new_data["ID_Target_Muscles"].apply(lambda x: x[index] == 1)]
            finished_data = pd.concat([finished_data, temporary_dataset], ignore_index=True)
    return finished_data


In [77]:
def adjust_score(predicted_scores, user_fitness_level, exercise_difficulty, max_difference=4):
    """
    Adjust scores based on the user's fitness level and exercise difficulty.
    """
    adjusted_scores = []
    for pred, difficulty in zip(predicted_scores, exercise_difficulty):
        factor = 1 - (abs(user_fitness_level - difficulty) / max_difference) ** 2
        factor = max(0.5, factor)
        adjusted_scores.append(pred * factor)
    return adjusted_scores

def knn(predict_data, user_fitness_level,All_Excercises):
    """
    Perform KNN classification and adjust the predicted scores.
    """
    training_data = original_training_data.copy()
    for col in ID_columns:
        def safe_literal_eval(value):
            try:
                # Ensure the value is parsed correctly
                if isinstance(value, str):
                    return np.array(ast.literal_eval(value))
                elif isinstance(value, list):
                    return np.array(value)
                else:
                    return np.array([])  # Return empty array for invalid entries
            except (ValueError, SyntaxError):
                return np.array([])  # Handle malformed data gracefully

        training_data[col] = training_data[col].apply(safe_literal_eval)

    # Flatten the training data
    flattened_features = []
    for col in ID_columns:
        arrays = np.stack(training_data[col].values)
        array_df = pd.DataFrame(arrays, columns=[f"{col}_{i}" for i in range(arrays.shape[1])])
        flattened_features.append(array_df)

    data_flattened = pd.concat([training_data.drop(ID_columns, axis=1)] + flattened_features, axis=1)

    X = data_flattened.drop(["Exercise Name", "Difficulty (1-5)", "score"], axis=1)
    y = data_flattened["score"]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_scaled, y)

    # Process predict_data
    for col in ID_columns:
        predict_data[col] = predict_data[col].apply(safe_literal_eval)

    # Flatten the test data
    flattened_features_test = []
    for col in ID_columns:
        arrays_test = np.stack(predict_data[col].values)
        array_df_test = pd.DataFrame(arrays_test, columns=[f"{col}_{i}" for i in range(arrays_test.shape[1])])
        flattened_features_test.append(array_df_test)

    data_test_flattened = pd.concat([predict_data.drop(ID_columns, axis=1)] + flattened_features_test, axis=1)

    X_test = data_test_flattened.drop(["Exercise Name", "Difficulty (1-5)"], axis=1)
    X_test_scaled = scaler.transform(X_test)

    y_pred_test = knn.predict(X_test_scaled)

    # Adjust scores using the function
    exercise_difficulty = data_test_flattened["Difficulty (1-5)"]
    adjusted_scores_test = adjust_score(y_pred_test, user_fitness_level, exercise_difficulty)

    data_test_flattened["Predicted Score"] = adjusted_scores_test

    sorted_results = data_test_flattened.sort_values(by="Predicted Score", ascending=False)
    sorted_results = sorted_results[~sorted_results['Exercise Name'].isin(All_Excercises[i])]
    return sorted_results["Exercise Name"].iloc[0]


In [78]:
# Välj relevanta kolumner för klustring
features_to_cluster = exercise_data[['Main_muscle', 'Target_Muscles']]

# Använd OneHotEncoder för att omvandla dessa kolumner till numeriska värden
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(features_to_cluster)

# Använd K-means för att klustra övningarna
kmeans = KMeans(n_clusters=10, random_state=42)  # Välj antal kluster till 11
exercise_data['Cluster'] = kmeans.fit_predict(encoded_features)

clustered_exercises = exercise_data.groupby('Cluster')['Exercise Name'].apply(list)

In [79]:
# Display the list of main muscles with numbers
def display_muscles(muscles):
    print("Select one or more main muscles from the list:")
    for index, muscle in enumerate(muscles, start=1):
        print(f"{index}. {muscle}")

# Select main muscles and filter target muscles
def select_MainMuscles(data):
    available_muscles = data['Main_muscle'].unique().tolist()  # List of available main muscles
    selected_muscles = set()  # Set to keep track of selected muscles for the current day

    selecting = True
    while selecting:
        # Display available main muscles
        print("Select one or more main muscles from the list:")
        for index, muscle in enumerate(available_muscles, start=1):
            print(f"{index}. {muscle}")

        choice = input("Enter the number of the muscle you want to select (or 'q' to finish selection): ")

        if choice.lower() == 'q':
            selecting = False
        
        else:
            try:
                choice_index = int(choice) - 1
                if 0 <= choice_index < len(available_muscles):
                    selected_muscle = available_muscles[choice_index]
                    if selected_muscle in selected_muscles:
                        print(f"You have already selected: {selected_muscle}. Please choose another muscle.")
                    else:
                        print(f"You have selected: {selected_muscle}")
                        selected_muscles.add(selected_muscle)  # Add to the set to prevent future selections for today
                else:
                    print("Invalid choice, please try again.")
            except ValueError:
                print("Please enter a valid number.")

    return selected_muscles  # Return the set of selected main muscles for the day

# Filter target muscles based on selected main muscles
def filter_TargetMuscles(data, selected_muscles):
    target_muscles = []  # List to store target muscles
    for muscle in selected_muscles:
        filtered_data = data[data['Main_muscle'] == muscle]
        for _, target in filtered_data.iterrows():
            target_name = target['Target_Muscles']  # Corrected column name
            if target_name not in target_muscles:  # Check for duplicates
                target_muscles.append(target_name)  # Append target muscles to the list

    return target_muscles  # Return the list of target muscles

# Function to handle pre-planned workouts
def PREplanned_workout():
    print("Select a pre-planned workout type:")
    print("1. Push")
    print("2. Pull")
    print("3. Leg")
    print("4. Whole Body")

    while True:
        choice = input("Enter the number of the workout type you want to select: ")

        try:
            choice_index = int(choice)
            if choice_index == 1:  # Push
                main_muscles = ['Chest', 'Shoulder', 'Triceps']
                print("You have selected: Push")
                return main_muscles
            elif choice_index == 2:  # Pull
                main_muscles = ['Back', 'Biceps', 'Forearm']
                print("You have selected: Pull")
                return main_muscles
            elif choice_index == 3:  # Leg
                main_muscles = ['Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Leg")
                return main_muscles
            elif choice_index == 4:  # Whole Body
                main_muscles = ['Chest', 'Back', 'Shoulder', 'Biceps', 'Triceps', 'Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Whole Body")
                return main_muscles
            else:
                print("Invalid choice, please try again.")
        except ValueError:
            print("Please enter a valid number.")

# Function to select difficulty level
def select_difficulty():
    print("Select difficulty level")
    while True:
        difficulty = input("Select difficulty level (1-5): ")
        try:
            difficulty = int(difficulty)
            if 1 <= difficulty <= 5:
                print(f"You have selected difficulty level: {difficulty}")
                return difficulty  # Return the selected difficulty level
            else:
                print("Please enter a number between 1 and 5.")
        except ValueError:
            print("Please enter a valid number.")

# Dictionary to store workouts for each day of the week
weekly_workouts = {}

# List of days in a week
days_of_week = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

# Ask for difficulty level once for the whole week
difficulty = select_difficulty()  # Get the difficulty level for the week

for day in days_of_week:
    
    print(f"{day}:")
    print("Select workout type:")
    print("1. Pre-planned")
    print("2. Hybrid")
    print("3. Rest Day")  # Added Rest Day option here

    while True:  # Loop until a valid choice is made
        workout_choice = input("Enter the number of the workout type you want to select: ")
        
        try:
            workout_choice_index = int(workout_choice)  # Try to convert input to an integer
            if workout_choice_index == 1:  # Pre-planned
                main_muscles = PREplanned_workout()
                break  # Exit the inner loop if a valid choice is made
            elif workout_choice_index == 2:  # Hybrid
                main_muscles = select_MainMuscles(exercise_data)
                break  # Exit the inner loop if a valid choice is made
            elif workout_choice_index == 3:  # Rest Day
                print("Rest day selected. No workout planned.")
                weekly_workouts[day] = []  # Set to an empty list for Rest Day
                break  # Exit the inner loop
            else:
                print("Invalid choice, please try again.")  # Prompt for valid input again
        except ValueError:
            print("Please enter a valid number.")  # Handle non-integer inputs
    
    # Collect all unique target muscles for the selected main muscles
    if workout_choice_index != 3:  # Only filter target muscles if it's not a Rest Day
        selected_targets = filter_TargetMuscles(exercise_data, main_muscles)

        # Store the selected target muscles in the dictionary
        if selected_targets:
            weekly_workouts[day] = selected_targets  # Store as a dictionary
    clear_output()


In [80]:
# Print the weekly workouts
print("Weekly Workouts:")
for day, exercises in weekly_workouts.items():
    muscles = {muscle.strip() for exercise in exercises for muscle in exercise.split(',') if muscle.strip() != ""}
    weekly_workouts[day] = list(muscles)
print(weekly_workouts)
print(f"Difficulty level for the week is set to: {difficulty}")

Weekly Workouts:
{'MONDAY': ['Supraspinatus', 'Serratus Anterior', 'Anterior Deltoid', 'Pectoralis Major Clavicular', 'Triceps Brachii', 'Pectoralis Major Sternal', 'Posterior Deltoid', 'Lateral Deltoid'], 'TUESDAY': ['Upper Trapezius', 'Brachioradialis', 'Biceps Brachii', 'Wrist Flexors', 'Wrist Extensors', 'Levator Scapulae', 'Infraspinatus', 'Rhomboids', 'Lower Trapezius', 'Subscapularis', 'Teres Minor', 'Brachialis', 'Supinator', 'Middle Trapezius', 'Latissimus Dorsi', 'Teres Major', 'Pronators'], 'WEDNESDAY': ['Gluteus Maximus', 'Quadriceps', 'Tibialis Anterior', 'Gastrocnemius', 'Soleus', 'Hamstrings'], 'THURSDAY': ['Gluteus Maximus', 'Quadriceps', 'Brachialis', 'Teres Minor', 'Serratus Anterior', 'Tibialis Anterior', 'Gastrocnemius', 'Subscapularis', 'Posterior Deltoid', 'Lateral Deltoid', 'Supraspinatus', 'Pectoralis Major Clavicular', 'Biceps Brachii', 'Rhomboids', 'Lower Trapezius', 'Triceps Brachii', 'Middle Trapezius', 'Latissimus Dorsi', 'Upper Trapezius', 'Anterior Deltoi

In [81]:
All_Excercises = {"MONDAY" :[], 
"TUESDAY": [], 
"WEDNESDAY": [], 
"THURSDAY": [], 
"FRIDAY": [], 
"SATURDAY": [], 
"SUNDAY": []}
for i in weekly_workouts:
    worked_muscles = []
    for k in weekly_workouts[i]:
        if worked_muscles.count(k) <= 2:
            k = k.replace(",", "") 
            predic_data = encoded_data(k)  
            excercise = knn(predic_data, difficulty, All_Excercises) 
            row = exercise_data[exercise_data['Exercise Name'] == excercise]  
            primary_muscles = row['Target_Muscles'].str.split(',').iloc[0] 
            worked_muscles.extend([muscle.strip() for muscle in primary_muscles if muscle.strip() != ""])
            secondary_muscles = row['Secondary Muscles'].str.split(',').iloc[0] 
            worked_muscles.extend([muscle.strip() for muscle in secondary_muscles if muscle.strip() != ""])
            All_Excercises[i].append(excercise)



In [82]:
# Lista över veckodagar för att underlätta indexering
days_of_week = list(All_Excercises.keys())

# Uppsättning för att hålla reda på använda övningar från föregående dag
previous_used_exercises = set()

# Funktion för att kontrollera om två övningar är för lika
def are_exercises_similar(exercise1, exercise2):
    # Jämför övningarnas namn, kan justeras för att vara mer eller mindre strikt
    return exercise1.lower() in exercise2.lower() or exercise2.lower() in exercise1.lower()

# Iterera över schemat
for i, day in enumerate(days_of_week):
    exercises = All_Excercises[day]
    #data.append({'Day': day, 'Exercise': '', 'Equipment': '', 'Preparation': '', 'Execution': ''})  # Blank rad för dagens namn
    used_exercises = set()  # Skapa en ny uppsättning för den aktuella dagen

    if exercises:  # Om det finns övningar för dagen
        for exercise in exercises:
            # Hämta information om övningen
            exercise_info = exercise_data.loc[exercise_data['Exercise Name'] == exercise]
            if not exercise_info.empty:
                exercise_name = exercise_info['Exercise Name'].values[0]  # Hämta övningsnamnet
                preparation = exercise_info['Preparation'].values[0] if 'Preparation' in exercise_info else ""
                execution = exercise_info['Execution'].values[0] if 'Execution' in exercise_info else ""
                equipment = exercise_info['Equipment'].values[0] if 'Equipment' in exercise_info else ""

                # Kontrollera om övningen redan har använts på föregående dag
                if i > 0 and (exercise_name in previous_used_exercises or any(are_exercises_similar(exercise_name, used_exercise) for used_exercise in previous_used_exercises)):
                    # Filtrera för att hitta en annan övning i samma kluster
                    cluster_number = exercise_info['Cluster'].values[0]  # Hämta klustret
                    alternative_exercises = exercise_data[exercise_data['Cluster'] == cluster_number]
                    alternative_exercises = alternative_exercises[~alternative_exercises['Exercise Name'].isin(used_exercises | previous_used_exercises)]

                    # Filtrera bort övningar som är för lika
                    alternative_exercises = alternative_exercises[~alternative_exercises['Exercise Name'].apply(lambda x: any(are_exercises_similar(x, used_exercise) for used_exercise in used_exercises | previous_used_exercises))]

                    if not alternative_exercises.empty:
                        # Välj en alternativ övning
                        alternative_exercise = alternative_exercises.sample(n=1)  # Välj slumpmässigt en övning
                        exercise_name = alternative_exercise['Exercise Name'].values[0]
                        preparation = alternative_exercise['Preparation'].values[0] if 'Preparation' in alternative_exercise else ""
                        execution = alternative_exercise['Execution'].values[0] if 'Execution' in alternative_exercise else ""
                        equipment = alternative_exercise['Equipment'].values[0] if 'Equipment' in alternative_exercise else ""

                # Lägg till övningen i used_exercises för den aktuella dagen
                used_exercises.add(exercise_name)

        # Efter att ha bearbetat övningarna för dagen, uppdatera previous_used_exercises
        previous_used_exercises = used_exercises.copy()





In [83]:
data = []

for day, exercises in All_Excercises.items():
    data.append({'Day': day, 'Exercise': '', 'Equipment': '', 'Preparation': '', 'Execution': ''})  # Blank row for the day name
    for exercise in exercises:
        information = exercise_data[exercise_data['Exercise Name'] == exercise]
        data.append({
            'Day': '',  # Leave the day column empty for subsequent exercises
            'Exercise': exercise,
            'Equipment': information['Equipment'].iloc[0],
            'Preparation': information['Preparation'].iloc[0],
            'Execution': information['Execution'].iloc[0]
        })
    data.append({'Day': '', 'Exercise': '', 'Equipment': '', 'Preparation': '', 'Execution': ''})

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head(70)

,Day,Exercise,Equipment,Preparation,Execution
0,MONDAY,,,,
1,,Front Lateral Raise,Cable,Grasp dumbbell cable attachment. Stand facing ...,"With elbow straight or slightly bent, raise up..."
2,,Incline Shoulder Raise,Barbell,Lie supine on incline bench. Dismount barbell ...,Raise shoulders toward bar as high as possible...
3,,Military Press: Seated,Barbell,Grasp barbell with slightly wider than shoulde...,Press bar upward until arms are extended overh...
4,,Incline Fly,Cable,Sit on seat with stirrups in each hand (attach...,Lower stirrups outward to sides of shoulders. ...
...,...,...,...,...,...
61,,,,,
62,SATURDAY,,,,
63,,,,,
64,SUNDAY,,,,


In [84]:
output_file = 'final_scheme.xlsx'
df.to_excel(output_file, index=False)